# Teams

Get List of NHL Teams

## Imports

In [81]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pyodbc

## Credentials

In [82]:
with open('credentials.csv', 'r') as f:

    userid, password = pd.read_csv(f, skipinitialspace=True)

## Get URL with Teams

In [83]:
def Connect_DB(database, userid, password):

    try:

        driver = '{ODBC Driver 17 for SQL Server}'
        query = 'DRIVER={};SERVER=linuxserver;DATABASE={};UID={};PWD={}'.format(driver, database, userid, password)

        cnxn = pyodbc.connect(query)
        cnxn.autocommit = True

        return cnxn

    except:
        
        return Null

In [84]:
def Get_Teams(url):

    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    conferences = soup.findAll('section', {'class': 'conference'})

    teams = list()

    for conference in conferences:

        conf = conference.h2.text.replace('Conference', '').strip()

        divisions = conference.findAll('div', {'class': 'division'})

        for division in divisions:

            div = division.h3.text.replace('Division', '').strip()

            team_cities = division.findAll('a', {'class': 'team-city'})
        
            for team_city in team_cities:

                city = team_city.find('span').text.strip()
                name = team_city.find('span').findNext('span').text.strip()

                listing = (city + ' ' + name, conf, div)

                teams.append(listing)

    return teams[:-1] # exclude Seattle for now

In [85]:
def Save_Data_to_Teams(teams):

    cursor = cnxn.cursor()
    
    if not cursor.tables(table='Teams', tableType='TABLE').fetchone():

        query = """CREATE TABLE Teams ( \
                       team varchar(50) PRIMARY KEY NOT NULL, \
                       conference varchar(50) NOT NULL, \
                       division varchar(50) NOT NULL)"""

        cursor.execute(query)

    try:

        cnxn.autocommit = False
        cursor.executemany("INSERT INTO Teams(team, conference, division) VALUES (?, ?, ?)", teams)

    except pyodbc.DatabaseError as err:

        #print (f'Error: {err}')
        cnxn.rollback()

    else:

        cnxn.commit()

    finally:

        cnxn.autocommit = True

## Main

In [86]:
if __name__ == '__main__':
    
    cnxn = Connect_DB('NHL', userid, password)
    url = pd.read_sql("SELECT [url] FROM URLs WHERE [use] = 'teams'", cnxn).values[0][0]
    
    teams = Get_Teams(url)
    
    Save_Data_to_Teams(teams)